In [7]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
import seaborn as sns
%matplotlib inline
from __future__ import print_function
import numpy as np
import statsmodels.api as sm
from scipy import stats
from matplotlib import pyplot as plt

In [8]:
percentage_change_all_vars = pd.read_csv("percentage_change_all_vars.csv")
percentage_change_all_vars.head()
newdf=percentage_change_all_vars[['UNITID','Rank','TYPE','0809_GR','0809_EN','0809_ASSETS','0809_EXPENSES','0809_REVENUE','0809_RESEARCH','0809_ANYAIDN','0809_LOAN_T', '0809_FGRNT_T','0809_SGRNT_T','0809_IGRNT_T']]
newdf['const']=1

/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [9]:
dummy_enrollment=[]
for index, row in newdf.iterrows():
    if row["0809_EN"]>0:
        dummy_enrollment.append(float(1))
    else:
        dummy_enrollment.append(float(0))
        
newdf['dummy_enrollment'] = dummy_enrollment

/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
dummiesfortype = pd.get_dummies(newdf['TYPE'])
data = pd.concat([newdf, dummiesfortype], axis=1)

from sklearn.utils import shuffle
data = shuffle(data)
data.drop(['TYPE','0809_EN'], 1, inplace=True)


Recursive Feature Elimination (RFE) is based on the idea to repeatedly construct a model and choose either the best or worst performing feature, setting the feature aside and then repeating the process with the rest of the features. This process is applied until all features in the dataset are exhausted. The goal of RFE is to select features by recursively considering smaller and smaller sets of features.

In [13]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn import datasets
from sklearn.linear_model import LinearRegression

logreg = LogisticRegression()
rfe = RFE(logreg, 18)
y=['dummy_enrollment']
listofcol=data.columns.values.tolist()
X=[i for i in listofcol[1:] if i not in y]
#X=['0809_EXPENSES_sq','0809_ANYAIDN_sq' ,'public']
print(X)
X_train, X_test, y_train, y_test = train_test_split(data[X],  data[y], test_size=0.3, random_state=0)
preprocessing.normalize(X_train, axis=0)
rfe = rfe.fit(X_train, y_train )
print(rfe.support_)
print(rfe.ranking_)

['Rank', '0809_GR', '0809_ASSETS', '0809_EXPENSES', '0809_REVENUE', '0809_RESEARCH', '0809_ANYAIDN', '0809_LOAN_T', '0809_FGRNT_T', '0809_SGRNT_T', '0809_IGRNT_T', 'const', 'for-profit', 'not-for-profit', 'public']
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#### import statsmodels.api as sm
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.575041
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:       dummy_enrollment   No. Observations:                 1106
Model:                          Logit   Df Residuals:                     1092
Method:                           MLE   Df Model:                           13
Date:                Fri, 27 Apr 2018   Pseudo R-squ.:                 0.07871
Time:                        19:57:56   Log-Likelihood:                -636.00
converged:                       True   LL-Null:                       -690.33
                                        LLR p-value:                 3.401e-17
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Rank              -0.0028      0.002     -1.228      0.219      -0.007       0.002
0809_GR          

/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1029: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.67


In [18]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=20, random_state=1)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, data[X],  data[y], cv=kfold, scoring=scoring)
print("20-fold cross validation average accuracy: %.3f" % (results.mean()))

/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

20-fold cross validation average accuracy: 0.681


/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vincentwahl/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

The result is telling us that we have 10872+254 correct predictions and 1122+109 incorrect predictions.

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[ 18 145]
 [ 11 301]]


The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier to not label a sample as positive if it is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

The F-beta score weights the recall more than the precision by a factor of beta. beta = 1.0 means recall and precision are equally important.

The support is the number of occurrences of each class in y_test.

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.62      0.11      0.19       163
        1.0       0.67      0.96      0.79       312

avg / total       0.66      0.67      0.59       475




https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8